In [90]:
import numpy as np
from itertools import product

# Classes for neurons

In [89]:
class SensoryNeuron:
    
    def __init__(
        self,
        eye,
        orientation,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.eye = eye
        self.orientation = orientation
        self.alpha = alpha
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.response = init_response
        self.habituation = init_habituation
        
    def one_timestep(
        self,
        suppressive_drive,
        dt
    ):
        _change_in_habituation = self._calculate_change_in_habituation(dt)
        _change_in_response = self._calculate_change_in_response(suppressive_drive, dt)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        sensory_input,
        snapshot
    ):
        #TO DO: implement how neuron gets information from snap + implement snap :D
        # compute sum over different opponency neurons
        self._excitatory_drive = rectification(
            sensory_input ** self.n - self.weight_opponency * opponency_response
        ) * rectification(
            1 + self.weight_attention * attention_response
        )
    
    def _calculate_change_in_response(
        self,
        suppressive_drive
    ):
        change_in_response = (
            - self.response + 
            (self.alpha * self.excitatory_drive) / 
            (suppressive_drive + self.habituation ** self.n + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response
        
    def _calculate_change_in_habituation(
        self
    ):
        change_in_habituation = (
            self.habituation + 
            self.weight_habituation * self.response
        ) * (dt / self.tau_habituation)
        return change_in_habituation

In [91]:
class SummationNeuron:
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        weight_habituation,
        tau_response,
        tau_habituation,
        init_response,
        init_habituation
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.tau_habituation = tau_habituation
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def one_timestep(
        self,
        dt
    ):
        _change_in_habituation = self._calculate_change_in_habituation(dt)
        _change_in_response = self._calculate_change_in_response(dt)
        self.response += _change_in_response
        self.habituation += _change_in_habituation
            
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
    ):
        #extract response left and right
        self._excitatory_drive = (response_left + response_right) ** self.n
        
    @property
    def suppressive_drive(
        self
    ):
        return self._excitatory_drive
    
    #option: def one_timestep
    def _calculate_change_in_response(
        self,
        dt
    ):
        change_in_response = (
            - self.response + self.excitatory_drive / 
            (self.suppressive_drive + self.habituation ** self.n + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response
    
    def _calculate_change_in_habituation(
        self,
        dt
    ):
        change_in_habituation = (
            self.habituation + 
            self.weight_habituation * self.response
        ) * (dt / self.tau_habituation)
        return change_in_habituation

In [62]:
class AttentionNeuron:
    
    def __init__(
        self,
        orientation,
        sigma,
        n,
        tau_reponse,
        init_response,
        init_habituation
    ):
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.weight_opponency = weight_opponency
        self.weight_attention = weight_attention
        self.weight_habituation = weight_habituation
        self.tau_response = tau_response
        self.init_response = init_response
        self.init_habituation = init_habituation
        
    def one_timestep(
        self
    ):
        pass
    
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        response_orientation_same,
        response_orientation_opposite
    ):
        self._excitatory_drive = ...

In [98]:
class OpponencyNeuron:
    
    def __init__(
        self,
        eye,
        orientation,
        sigma,
        n,
        tau_response,
        init_response
    ):
        self.eye = eye
        self.orientation = orientation
        self.sigma = sigma
        self.n = n
        self.tau_response = tau_response
        self.init_response = init_response
        
    def one_timestep(
        self,
        suppressive_drive,
        dt
    ):
        _change_in_response = self.calculate_change_in_response(suppressive_drive, dt)
        self.response += _change_in_response
        
    @property
    def excitatory_drive(
        self
    ):
        return self._excitatory_drive
    
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        #extract the correct signals from snap
        self._excitatory_drive = rectification(response_same_eye - response_other_eye) ** self.n
        
    def calculate_change_in_response(
        self,
        suppressive_drive,
        dt
    ):
        change_in_response = (
            - self.response +
            self.excitatory_drive /
            (suppressive_drive + self.sigma ** self.n)
        ) * (dt / self.tau_response)
        return change_in_response

# Classes for populations

class Population:
    
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        for neuron in neurons.values():
            neuron.compute_excitatory_drive(snapshot)

In [79]:
class SensoryPopulation(Population):
    
    eyes = ['left', 'right']
    orientations = [1, 2]
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SensoryNeuron(
                eye=eye,
                orientation=orientation,
                alpha=alpha,
                sigma=sigma,
                n=n,
                weight_opponency=weight_opponency,
                weight_attention=weight_attention,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def compute_excitatory_drive(
        self,
        sensory_input,
        snap
    ):
        for neuron in neurons.values():
            neuron.compute_excitatory_drive(sensory_input, snap)
        
    def one_timestep(
        self,
        sensory_input,
        snap
    ):
        for neuron in neurons.values():
            neuron.one_timestep(self.suppressive_drive, dt)
        
    @property
    def suppressive_drive(self):
        return sum([neuron.excitatory_drive for neuron in self.neurons.values()])

In [92]:
class SummationPopulation:
    
    orientations = [1, 2]
    
    def __init__(
        self,
        sigma,
        n,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:    
                _initial_habituation = np.random.rand()
            self.neurons[key] = SummationNeuron(
                orientation=orientation,
                sigma=sigma,
                n=n,
                weight_habituation=weight_habituation,
                tau_response=tau_response,
                tau_habituation=tau_habituation,
                initial_response=initial_response,
                initial_habituation=initial_habituation
            )
            
    def compute_excitatory_drive(
        self,
        snapshot
    ):
        for neuron in self.neurons.values():
            neuron.compute_excitatory_drive(snapshot)
            
    def one_timestep(
        self
    ):
        for orientation in self.orientations:
            self.neurons[orientation].one_timestep()

In [79]:
class OpponencyPopulation:
    
    eyes = ['left', 'right']
    orientations = [1, 2]
    
    def __init__(
        self,
        sigma,
        n,
        tau_response,
        initial_response=None,
    ):
        for eye, orientation in product(self.eyes, self.orientations):
            key = eye + '_' + orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            self.neurons[key] = OpponencyNeuron(
                eye=eye,
                orientation=orientation,
                sigma=sigma,
                n=n,
                tau_response=tau_response,
                initial_response=initial_response,
            )
            
    def compute_excitatory_drives(
        self,
        snapshot
    ):
        for neuron in self.neurons.values():
            neuron.compute_excitatory_drive(snapshot)
        
    def one_timestep(
        self,
        dt
    ):
        suppressive_drives = self.suppressive_drives
        for eye in self.eyes:
            suppressive_drive_eye = suppressive_drives[eye]
            for orientation in self.orientations:
                self.neurons[orientation].one_timestep(suppressive_drive_eye, dt)
                
    @property
    def suppressive_drives(self):
        drives = {}
        for eye in self.eyes:
            drives[eye] = sum([neurons[f'{eye}_{i}'].excitatory_drive for i in self.orientations])
        return drives

In [81]:
class AttentionPopulation:
    
    orientations = [1,2]
    
    def __init__(
        self,
        alpha,
        sigma,
        n,
        weight_opponency,
        weight_attention,
        weight_habituation,
        tau_response,
        tau_habituation, 
        initial_response=None,
        initial_habituation=None
    ):
        for orientation in self.orientations:
            key = orientation
            if key in initial_response.keys():
                _initial_response = initial_reponse[key]
            else:
                _initial_response = np.random.rand()
            if key in initial_habituation.keys():
                _initial_habituation = initial_habituation[key]
            else:
                _initial_habituation = np.random.rand()
            self.neurons[key] = AttentionNeuron(
                orientation=orientation,
                sigma=sigma,
                n=n,
                tau_response=tau_response,
                initial_response=initial_response,
                initial_habituation=initial_habituation               
            )
            
    

# Utilities

In [ ]:
@dataclass
class Snapshot:
        

In [87]:
def get_rectification(smooth=True):
    if smooth:
        def _rectification():
            print('smooth')
    else:
        def _rectification():
            print('not smooth')
        
    return _rectification
rectification = get_rectification()

# General structure

In [ ]:
sensory_population = [SensoryNeuron('right', 1), SensoryNeuron('...')]
attention_population = [...]

#define input
condition = 'switch_stimuli'
stimulus_input = get_input(condition)

def get_input(condition, n_trials):
    if condition == 'switch_stimuli':
        return np.array([0,1] * (n_trials // 2))
    else:
        raise 
        
#init_firing_rate
class Snapshot:
    
    def __init__(self, populations):
        ...
def extract_results(snapshots):
    pass

snaps = []
t_steps = 10000
t_delta = .01
for i in range(t_steps):
    current_time = t_delta * i
    c_input = stimulus_input[i]
    snap.append(Snapshot(sensory_population, attention_Layer, opponency_population))
    sensory_population.update(snap)
    for neuron in sensory_population:
        neuron.update_weights(c..)
    attention_population.update_weights(snap[-1])
    opponency_population.update_weights(snap[-1])
    
results = extract_results(snap)

## Plan für morgen:
 * Summation-Neurons / Summation-Population
 * Opponency-Neurons / Opponency-Population
 * Attention-Neurons / Attention-Population
 * Snapshot
 * Snapshot -> Results